In [1]:
import pandas as pd

df = pd.read_csv('accessPerDay.csv')
df

,Name,Group,Date,Duration
0,Samuel Flowers,T02,2020-11-15,938.900000
1,Samuel Flowers,T02,2020-11-14,1287.916667
2,Samuel Flowers,T02,2020-11-13,745.716667
3,Samuel Flowers,T02,2020-11-12,645.083333
4,Samuel Flowers,T02,2020-11-11,359.500000
...,...,...,...,...
958,Leighton Mack,T03,2020-09-22,239.783333
959,Evelyn Bull,R01,2020-09-22,126.033333
960,Hisham Avalos,T01,2020-09-15,125.750000
961,Sadie Hunt,T03,2020-10-04,52.000000


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 963 entries, 0 to 962
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      963 non-null    object 
 1   Group     963 non-null    object 
 2   Date      963 non-null    object 
 3   Duration  963 non-null    float64
dtypes: float64(1), object(3)
memory usage: 30.2+ KB


In [3]:
list(df)

['Name', 'Group', 'Date', 'Duration']

# Q2a

In [4]:
import sqlite3
import csv

conn = sqlite3.connect('students.db')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS submission_log')
cur.execute('CREATE TABLE submission_log (Name TEXT, "Group" TEXT, Date TEXT, Duration REAL)')
conn.commit()

rows = []
count = 0

with open('accessPerDay.csv') as file:
    csv_reader = csv.reader(file)
    headers = next(csv_reader)
    
    for row in csv_reader:
        rows.append(tuple(row))
        count += 1
        
cur.executemany('INSERT INTO submission_log (Name, "Group", Date, Duration) VALUES (?, ?, ?, ?)', rows)

conn.commit()
conn.close()

In [5]:
conn = sqlite3.connect('students.db')
cur = conn.cursor()

cur.execute('SELECT COUNT(*) FROM submission_log')

assert count == cur.fetchone()[0] # Validate all rows are inserted

conn.close()

# Q2b(i)

In [6]:
conn = sqlite3.connect('students.db')
cur = conn.cursor()

cur.execute('SELECT "Group", AVG(Duration) FROM submission_log GROUP BY "Group" ORDER BY Duration DESC')

for record in cur.fetchall():
    print(f"{record[0]} - {record[1]}")
    
conn.close()

T02 - 359.86715506811356
T01 - 352.96365313715506
T03 - 276.8282505895036
R01 - 63.5266666666


# Q2b(ii)

In [7]:
conn = sqlite3.connect('students.db')
cur = conn.cursor()

cur.execute('SELECT Name, SUM(Duration) FROM submission_log GROUP BY Name ORDER BY SUM(Duration) Desc')

print("Top 3 most active students")
for record in cur.fetchall()[:3]:
    print(f"{record[0]} - {record[1]}")

Top 3 most active students
Karim Duggan - 17991.683333629997
Samuel Flowers - 17442.26666727
Aleksander Tucker - 16283.016666739999


In [8]:
cur.execute('SELECT Name, SUM(Duration) FROM submission_log GROUP BY Name ORDER BY SUM(Duration) Desc')

print("Top 3 least active students")
for record in cur.fetchall()[-3:]:
    print(f"{record[0]} - {record[1]}")
    
conn.close()

Top 3 least active students
Sadie Hunt - 52.0
Ainsley Fry - 28.81666667
Dion Huff - 4.733333333
